# Library

In [ ]:
!pip install demoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd, numpy as np
import demoji, re
from string import punctuation

# Load Dataset

In [ ]:
df = pd.read_excel('/content/Dataset_VIKOR.xlsx')
ky = pd.read_excel('/content/Keyword_KeyBERT.xlsx')
df_kamus = pd.read_csv('kamus_alay.csv')
kamus = dict(zip(df_kamus['slang'], df_kamus['formal'])) # kamus alay ke baku

# Majority vote (ambil mode per baris)
df['emosi'] = df[['emosi_sihab', 'emosi_rijal', 'emosi_naufal']].mode(axis=1)[0]

# Mapping label emosi ke angka
label_map = {'anger': 3, 'fear': 2, 'sadness': 1}
df['emosi_encoded'] = df['emosi'].map(label_map)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   keluhan_baku     1499 non-null   object
 1   emosi_sihab      1500 non-null   object
 2   emosi_rijal      1500 non-null   object
 3   emosi_naufal     1500 non-null   object
 4   keyword_sihab    1500 non-null   object
 5   keyword_naufal   1500 non-null   object
 6   keyword_keybert  1500 non-null   object
 7   emosi            1500 non-null   object
 8   emosi_encoded    1500 non-null   int64 
dtypes: int64(1), object(8)
memory usage: 105.6+ KB


# Preprocesing Keyword

In [ ]:
def process_text(text):
    text = str(text) # Convert Object to str
    text = text.lower() # Lowercase text
    text = re.sub(r'\d+', '', text) # Remove number
    text = text.replace('\\n\\n\\n', ' ')
    text = text.replace('\\n\\n', ' ')
    text = text.replace('\\n', ' ')
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE) # Remove link
    text = re.sub(f"[{re.escape(punctuation)}]", " ", text) # Remove punctuation
    text = demoji.replace(text, "") # Remove emoji
    text = " ".join(text.split()) # Remove extra spaces, tabs, and new lines
    text = text.split()
    text = [kamus[word] if word in kamus else word for word in text]
    text = ' '.join(text)

    return text

ky['keyword'] = ky['keyword'].apply(process_text)
ky[ky['frequency'] >= 10]

,keyword,frequency,rank
0,mbr,755,1
1,pju,699,2
2,jalan,660,3
3,mohon,585,4
4,kelurahan,553,5
...,...,...,...
985,krampung,10,986
986,roda,10,987
987,kerumah,10,988
988,saran,10,989


# Perhitungan VIKOR

In [ ]:
# Fungsi untuk menghitung rata-rata ranking keyword dalam satu baris
def hitung_rata_rata(row):
    total_ranking = 0
    total_keyword = 0
    for keyword in row:
        frekuensi_rank = ky.loc[ky['keyword'] == keyword]
        if not frekuensi_rank.empty:
            total_ranking += frekuensi_rank['rank'].values[0]
            total_keyword += 1
    if total_keyword > 0:
        return total_ranking / total_keyword
    else:
        return 9999

df['rata_rata_ranking'] = df['keyword_keybert'].apply(hitung_rata_rata)

In [ ]:
f_emosi_plus = df['emosi_encoded'].max()
f_emosi_min = df['emosi_encoded'].min()
f_ranking_plus = df['rata_rata_ranking'].max()
f_ranking_min = df['rata_rata_ranking'].min()
df['new_keyword'] = f_ranking_plus - df['rata_rata_ranking'] + 1

In [ ]:
df['normalisasi_emosi'] = (f_emosi_plus - df['emosi_encoded'])/(f_emosi_plus - f_emosi_min)
df['normalisasi_ranking'] = (f_ranking_plus - df['new_keyword'])/(f_ranking_plus - f_ranking_min)

In [ ]:
df['normalisasi_bobot_emosi'] = 0.5 * df['normalisasi_emosi']
df['normalisasi_bobot_ranking'] = 0.5 * df['normalisasi_ranking']

In [ ]:
df['ultility'] = df['normalisasi_bobot_emosi'] + df['normalisasi_bobot_ranking']
df['regret'] = df[['normalisasi_bobot_emosi', 'normalisasi_bobot_ranking']].max(axis=1)

In [ ]:
s_plus = df['ultility'].max()
s_min = df['ultility'].min()
r_plus = df['regret'].max()
r_min = df['regret'].min()

In [ ]:
df['vikor'] = 0.5*((df['ultility']-s_min)/(s_plus-s_min)) + (1-0.5)*(df['regret']-r_min)/(r_plus-r_min)
df.head()

,keluhan_baku,emosi_sihab,emosi_rijal,emosi_naufal,keyword_sihab,keyword_naufal,keyword_keybert,emosi,emosi_encoded,rata_rata_ranking,new_keyword,normalisasi_emosi,normalisasi_ranking,normalisasi_bobot_emosi,normalisasi_bobot_ranking,ultility,regret,vikor
0,proyek pembangunan rumah tinggal membakar samp...,anger,anger,anger,"['asap', 'mengganggu', 'membakar', 'mencemari'...","['limbah', 'mencemari', 'mengganggu', 'tindakl...","['mengganggu', 'asap', 'mencemari', 'limbah', ...",anger,3,3964.055556,6035.944444,0.0,0.479747,0.0,0.239874,0.239874,0.239874,0.233479
1,ass wr wb selamat siang yth kepala dinas sumbe...,sadness,sadness,sadness,"['pavingisasi', 'hujan', 'banjir', 'aspal', 't...","['memprihatinkan', 'banjir', 'rusak', 'tidak l...","['banjir', 'pembangunan', 'ardinanta', 'paving...",sadness,1,3583.540000,6416.460000,1.0,0.433684,0.5,0.216842,0.716842,0.500000,0.850357
2,1 kartu keluarga nama yang terdaftar mbr des 2...,anger,anger,anger,"['nama', 'tidak terdaftar', 'mbr', 'KK', nan]","['tidak terdaftar', 'data MBR', 'cek', 'menany...","['2021', 'keluarga', 'mbr', 'kartu', 'semua', ...",anger,3,3050.730769,6949.269231,0.0,0.369185,0.0,0.184592,0.184592,0.184592,0.136787
3,pandemi maret 2020 sudah mengurus mbr 4 kali b...,sadness,sadness,anger,"['pandemi', 'mengurus', 'mbr', 'tidak berhasil...","['tidak berhasil', 'pandemi', 'mengurus', 'MBR...","['mbr', 'subhakti', 'yatim', '2020', 'pandemi'...",sadness,1,3544.777778,6455.222222,1.0,0.428992,0.5,0.214496,0.714496,0.500000,0.848605
4,mohon dibantu pembersihan waduk kebraon sebera...,sadness,sadness,fear,"['pembersihan', 'waduk', 'sampah', 'numpuk', nan]","['mohon', 'dibantu', 'pembersihan', 'sampah', ...","['karangpilang', 'waduk', 'eceng', 'sampah', '...",sadness,1,3500.687500,6499.312500,1.0,0.423654,0.5,0.211827,0.711827,0.500000,0.846612


In [40]:
df['rank'] = df['vikor'].rank(ascending=True).astype(int)
df['status'] = 'selesai'
df

,keluhan_baku,emosi_sihab,emosi_rijal,emosi_naufal,keyword_sihab,keyword_naufal,keyword_keybert,emosi,emosi_encoded,rata_rata_ranking,new_keyword,normalisasi_emosi,normalisasi_ranking,normalisasi_bobot_emosi,normalisasi_bobot_ranking,ultility,regret,vikor,rank,status
0,proyek pembangunan rumah tinggal membakar samp...,anger,anger,anger,"['asap', 'mengganggu', 'membakar', 'mencemari'...","['limbah', 'mencemari', 'mengganggu', 'tindakl...","['mengganggu', 'asap', 'mencemari', 'limbah', ...",anger,3,3964.055556,6035.944444,0.0,0.479747,0.0,0.239874,0.239874,0.239874,0.233479,415,selesai
1,ass wr wb selamat siang yth kepala dinas sumbe...,sadness,sadness,sadness,"['pavingisasi', 'hujan', 'banjir', 'aspal', 't...","['memprihatinkan', 'banjir', 'rusak', 'tidak l...","['banjir', 'pembangunan', 'ardinanta', 'paving...",sadness,1,3583.540000,6416.460000,1.0,0.433684,0.5,0.216842,0.716842,0.500000,0.850357,1120,selesai
2,1 kartu keluarga nama yang terdaftar mbr des 2...,anger,anger,anger,"['nama', 'tidak terdaftar', 'mbr', 'KK', nan]","['tidak terdaftar', 'data MBR', 'cek', 'menany...","['2021', 'keluarga', 'mbr', 'kartu', 'semua', ...",anger,3,3050.730769,6949.269231,0.0,0.369185,0.0,0.184592,0.184592,0.184592,0.136787,44,selesai
3,pandemi maret 2020 sudah mengurus mbr 4 kali b...,sadness,sadness,anger,"['pandemi', 'mengurus', 'mbr', 'tidak berhasil...","['tidak berhasil', 'pandemi', 'mengurus', 'MBR...","['mbr', 'subhakti', 'yatim', '2020', 'pandemi'...",sadness,1,3544.777778,6455.222222,1.0,0.428992,0.5,0.214496,0.714496,0.500000,0.848605,1081,selesai
4,mohon dibantu pembersihan waduk kebraon sebera...,sadness,sadness,fear,"['pembersihan', 'waduk', 'sampah', 'numpuk', nan]","['mohon', 'dibantu', 'pembersihan', 'sampah', ...","['karangpilang', 'waduk', 'eceng', 'sampah', '...",sadness,1,3500.687500,6499.312500,1.0,0.423654,0.5,0.211827,0.711827,0.500000,0.846612,1053,selesai
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,assalamualaikum warahmatullahi wabarakatuh tol...,sadness,sadness,sadness,"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]","['warahmatullahi', 'wabarakatuh', 'mbr', 'pend...",sadness,1,3040.102041,6959.897959,1.0,0.367898,0.5,0.183949,0.683949,0.500000,0.825798,820,selesai
1496,orang tua anak nama irsya sakinah mawadah rahm...,sadness,sadness,sadness,"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]","['pawiyatan', 'anak', 'sakinah', 'sekolah', 'k...",sadness,1,3597.617021,6402.382979,1.0,0.435388,0.5,0.217694,0.717694,0.500000,0.850993,1129,selesai
1497,tolong keluhan kemarin ada balas ya maturnuwun,anger,sadness,anger,"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]","['maturnuwun', 'balas', 'keluhan', 'tolong', '...",anger,3,3688.320000,6311.680000,0.0,0.446368,0.0,0.223184,0.223184,0.223184,0.204287,278,selesai
1498,mohon pengajuan permakanan untuk lansia a n tr...,anger,sadness,anger,"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]","['rt04', '3578065805620001', 'trisnowati', 'ti...",anger,3,3934.533333,6065.466667,0.0,0.476174,0.0,0.238087,0.238087,0.238087,0.230353,405,selesai


In [41]:
df.to_excel('vikor_fix.xlsx', index = False)